<h1 id="setup" align="center" style="color:white; background-color:#801A9A; padding:10px; border-bottom: 3px solid #460E54;">Setup</h1>

In [1]:
pip install opencv-python


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pytorch-fid


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install paramiko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 2.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 6.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 1.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 1.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import shutil
import pathlib
from PIL import Image

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
import plotly.express as px

from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torchvision.io import read_image
import torch.optim as optim

from tqdm import tqdm  # Import tqdm for progress bar
import warnings

Using PyTorch version 2.3.0


device(type='cuda')

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

In [ ]:
warnings.filterwarnings('ignore')
print('Using PyTorch version', torch.__version__)

# Setup device-agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

<h1 id="initiating-image-loading" align="center" style="color:white; background-color:#801A9A; padding:10px; border-bottom: 3px solid #460E54;">Initiating Image Loading</h1>

In [ ]:
attr_df = pd.read_csv('path/to/list_attr_celeba.csv')
bbox_df = pd.read_csv('path/to/list_bbox_celeba.csv')
eval_df = pd.read_csv('path/to/list_eval_partition.csv')
landmarks_df = pd.read_csv('path/to/list_landmarks_align_celeba.csv')

# Merge dataframes on the image_id
df = attr_df.merge(bbox_df, on='image_id')
df = df.merge(eval_df, on='image_id')
df = df.merge(landmarks_df, on='image_id')

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

In [ ]:
# Define transformations
transformations = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

In [ ]:
# Define the Custom Dataset class
class CelebADataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_filename = self.dataframe.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_filename)
        image = Image.open(img_path).convert('RGB')
        
        # Get attributes
        labels = self.dataframe.iloc[idx, 1:41].values.astype(float)
        labels = torch.tensor(labels, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

<h1 id="Visualizing" align="center" style="color:white; background-color:#801A9A; padding:10px; border-bottom: 3px solid #460E54;">Visualizing</h1>

In [ ]:
# Utility function to visualize some images
def visualize_samples(dataset, num_samples=5):
    fig, axs = plt.subplots(1, num_samples, figsize=(15, 15))
    for i in range(num_samples):
        idx = np.random.randint(0, len(dataset))
        img, labels = dataset[idx]
        img = img.permute(1, 2, 0).numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        axs[i].imshow(img)
        axs[i].set_title(f'Labels: {labels.numpy()}')
        axs[i].axis('off')
    plt.show()

<h1 id="Model Setup & Development" align="center" style="color:white; background-color:#801A9A; padding:10px; border-bottom: 3px solid #460E54;">Model Setup & Development</h1>

In [ ]:
# Initialize the dataset
img_dir = 'path/to/img_align_celeba'
dataset = CelebADataset(df, img_dir, transform=transformations)

# Split the dataset
train_df = df[df['partition'] == 0]
val_df = df[df['partition'] == 1]
test_df = df[df['partition'] == 2]

train_dataset = CelebADataset(train_df, img_dir, transform=transformations)
val_dataset = CelebADataset(val_df, img_dir, transform=transformations)
test_dataset = CelebADataset(test_df, img_dir, transform=transformations)

# Data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Check the number of samples in each set
print(f'Training set size: {len(train_dataset)}')
print(f'Validation set size: {len(val_dataset)}')
print(f'Test set size: {len(test_dataset)}')

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

<div style="color:#5C5B5B; border-right: 3px solid #5C5B5B; text-align: right; padding: 0px 7px; font-size: 1.17em; font-weight: bold;">Creating Generator & Discriminator</div>

In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, noise_dim=100, img_channels=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(noise_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

class Discriminator(nn.Module):
    def __init__(self, img_channels=3):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(img_channels, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False)
        )

    def forward(self, x):
        return self.main(x).view(-1)

# Initialize models
noise_dim = 100
generator = Generator(noise_dim).to(device)
discriminator = Discriminator().to(device)

print(generator)
print(discriminator)

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

<div style="color:#5C5B5B; border-right: 3px solid #5C5B5B; text-align: right; padding: 0px 7px; font-size: 1.17em; font-weight: normal;"><strong>Optimizers for both</strong> (Generator & Discriminator) and <strong>Gradient penalty function</strong> (for WGAN-GP)</div>


In [ ]:
lr = 0.0002
beta1 = 0.5
beta2 = 0.999

optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

# Gradient penalty function for WGAN-GP
def gradient_penalty(discriminator, real_samples, fake_samples):
    alpha = torch.randn(real_samples.size(0), 1, 1, 1, device=device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = discriminator(interpolates)
    fake = torch.ones(d_interpolates.size(), requires_grad=False, device=device)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>


<h1 id="Training and Optimization" align="center" style="color:white; background-color:#801A9A; padding:10px; border-bottom: 3px solid #460E54;">Training and Optimization</h1>

<span style="color:#5C5B5B; font-size:1.17em; font-style:italic; border-top:1px solid #5C5B5B; display:block; padding: 0px 5px;">Explanation:</span>

<h1 id="Evaluation" align="center" style="color:white; background-color:#801A9A; padding:10px; border-bottom: 3px solid #460E54;">Evaluation</h1>